# Linear Regression Consulting Project


    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    

In [1]:
import findspark
findspark.init('/home/coneylu/spark-3.0.0-preview2-bin-hadoop3.2')
import pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Hyundai').getOrCreate()

In [3]:
data = spark.read.csv('cruise_ship_info.csv',header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
#Check missing value 
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
rows = data.count()
summary = data.describe().filter(col('summary') == 'count')
summary.select(*((lit(rows)-col(c)).alias(c) for c in data.columns)).show()

+---------+-----------+---+-------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+-------+----------+------+------+-----------------+----+
|      0.0|        0.0|0.0|    0.0|       0.0|   0.0|   0.0|              0.0| 0.0|
+---------+-----------+---+-------+----------+------+------+-----------------+----+



In [6]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [9]:
#Vector and VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [10]:
assembler = VectorAssembler(inputCols = ['Age','Tonnage','passengers','length','cabins','passenger_density'],
                           outputCol = 'features')
output = assembler.transform(data)

In [13]:
output.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64])),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [17]:
final_data = output.select('features','crew')

In [18]:
train, test = final_data.randomSplit([0.8,0.2]) #split for 80%, 20%

In [19]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train) #fit training data

In [21]:
test_result = lr_model.evaluate(test)

In [22]:
test_result.rootMeanSquaredError

0.6854667001812068

In [23]:
test_result.r2

0.9420342492552505

In [34]:
#check correlation with each feature
from pyspark.sql.functions import corr
data.select(corr('crew','cabins')).collect()

[Row(corr(crew, cabins)=0.9508226063578497)]

In [35]:
data.select(corr('crew','Tonnage')).collect()

[Row(corr(crew, Tonnage)=0.927568811544939)]

In [36]:
data.select(corr('crew','passengers')).collect()

[Row(corr(crew, passengers)=0.9152341306065384)]